In [1]:
#modul de carrega i de neteja de les dades, comú des de la primera pràctica

%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

#per realitzar les grafiques de regrassió ha estat necessari ampliar els chunks k

plt.rcParams["agg.path.chunksize"]=10000

#carreguem els 3 camps en un de sol per poder utilitzar-el als analisi
df = pd.read_csv("DelayedFlights.csv", parse_dates= {"Data" : ["Year","Month","DayofMonth"]})

#en la pràctica inicial es demanar carregar sols uns columnes en concret per poder analitzar, en base comentaris
#carrego totes les columnes
#df=DelayFligths_df[["Data","DayOfWeek","UniqueCarrier","Origin","Dest","AirTime","Distance","ArrDelay","DepDelay","DepTime","ArrTime","CancellationCode"]]

#millorat el procés de neteja / analisi de la qualitat en les dades
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)


df["ArrDelay"] = df['ArrDelay'].fillna(df['ArrDelay'].mean())

df["DepDelay"] = df['DepDelay'].fillna(df['DepDelay'].mean())

df["AirTime"] = df['AirTime'].fillna(df['AirTime'].mean())
df["Distance"] = df['Distance'].fillna(df['Distance'].mean())


missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)


df.dropna(inplace = True)

#esborren totes les files que segueixin contenint algun null, tot i la perdua dinformació
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)





,variable,missing values,filling factor (%)
0,Data,0,100.0
1,NASDelay,0,100.0
2,WeatherDelay,0,100.0
3,CarrierDelay,0,100.0
4,Diverted,0,100.0
5,CancellationCode,0,100.0
6,Cancelled,0,100.0
7,TaxiOut,0,100.0
8,TaxiIn,0,100.0
9,Distance,0,100.0


In [2]:
#bloc per transformar les dades descriptives en valors numerics
# i ajustar les escales

msk = np.random.rand(len(df)) < 0.8

train = df[msk]

test = df[~msk]



#transformo les dades origen i destí
origin_cod = df[['Origin']]
desti_cod = df[['Dest']]


ordinal_encoder = OrdinalEncoder()
df["origin_enc"] = ordinal_encoder.fit_transform(origin_cod)

ordinal_encoder = OrdinalEncoder()
desti_cod_encoded = ordinal_encoder.fit_transform(desti_cod)
df["desti_enc"] = ordinal_encoder.fit_transform(desti_cod)
print(desti_cod_encoded[:10])

#escalo els valors de temps arribada i temps de soritda perquè quedin els valors entorn de 0 


min_max_scaler = MinMaxScaler()
deptime_values = df[['DepTime']]
scaled_values = min_max_scaler.fit(deptime_values)
df["deptime_esc"]=min_max_scaler.transform(deptime_values)

min_max_scaler = MinMaxScaler()
arrtime_values = df[['ArrTime']]
scaled_values = min_max_scaler.fit(arrtime_values)
df["arrtime_esc"]=min_max_scaler.transform(arrtime_values)


pd.get_dummies(df, columns=["CancellationCode"])




[[ 48.]
 [155.]
 [177.]
 [220.]
 [284.]
 [ 48.]
 [ 48.]
 [104.]
 [177.]
 [179.]]


,Data,Unnamed: 0,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,...,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,origin_enc,desti_enc,deptime_esc,arrtime_esc,CancellationCode_N
3,2008-01-03,4,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,...,2.0,0.0,0.0,0.0,32.0,140.0,48.0,0.761984,0.816173,1
5,2008-01-03,6,4,1937.0,1830,2037.0,1940,WN,509,N763SW,...,10.0,0.0,0.0,0.0,47.0,140.0,155.0,0.807003,0.848687,1
7,2008-01-03,11,4,1644.0,1510,1845.0,1725,WN,1333,N334SW,...,8.0,0.0,0.0,0.0,72.0,140.0,177.0,0.684869,0.768654,1
9,2008-01-03,16,4,1452.0,1425,1640.0,1625,WN,675,N286WN,...,3.0,0.0,0.0,0.0,12.0,140.0,220.0,0.604835,0.683201,1
11,2008-01-03,18,4,1323.0,1255,1526.0,1510,WN,4,N674AA,...,0.0,0.0,0.0,0.0,16.0,140.0,284.0,0.551063,0.635682,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936751,2008-12-13,7009705,6,921.0,830,1112.0,1008,DL,1616,N907DE,...,51.0,0.0,13.0,0.0,0.0,18.0,215.0,0.383493,0.463110,1
1936752,2008-12-13,7009709,6,1552.0,1520,1735.0,1718,DL,1620,N905DE,...,0.0,0.0,0.0,0.0,17.0,133.0,18.0,0.646519,0.722801,1
1936753,2008-12-13,7009710,6,1250.0,1220,1617.0,1552,DL,1621,N938DL,...,3.0,0.0,0.0,0.0,22.0,199.0,18.0,0.520634,0.673614,1
1936754,2008-12-13,7009717,6,657.0,600,904.0,749,DL,1631,N3743H,...,0.0,57.0,18.0,0.0,0.0,241.0,18.0,0.273447,0.376407,1


In [3]:
# EXERCICI 2, calculem els 3 models de regressió i després valorem segons MSE i al R2
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error as mae
import math
#regressió lineal, es calcula la relació entre la variable Arrdelay depenent del  temps de vol 
#df2_x = pd.DataFrame(np.c_[df['AirTime']], columns = ['AirTime'])

df2_x = pd.DataFrame(df.AirTime)

df2_y = pd.DataFrame(df.ArrDelay)

x_train, x_test, y_train, y_test = train_test_split(df2_x, df2_y, test_size=0.33, random_state=42)


lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)


#R2 i MSE de la regrecció lineal
#prediccióamb les dades de train
y_train_pred = lin_reg.predict(x_train)

#predicció amb les dades de test
y_test_pred = lin_reg.predict(x_test)

# AVALUANT AMB LES DADES DE TRAIN
# calculant manualment diferències entre les dades de train originals i les realitzaces en predicció

d = y_train - y_train_pred
mse_f = np.mean(d**2)
mae_f = np.mean(abs(d))
rmse_f = np.sqrt(mse_f)
r2_f = 1-(np.sum(d**2)/np.sum((y_train-np.mean(y_train))**2))

print("Result del calcul manual:")
print("MAE:",mae_f)
print("MSE:", mse_f)
print("RMSE:", rmse_f)
print("R-Squared:", r2_f)

#
# Calculant amb les funcions 
#
print('MSE conjunt train: %.3f' % (mean_squared_error(y_train, y_train_pred)))
#
# R-Squared
#
print('R^2 conjunt train: %.3f' % (r2_score(y_train, y_train_pred)))


# AVALUANT AMB LES DADES DE test

d = y_test - y_test_pred
mse_f = np.mean(d**2)
mae_f = np.mean(abs(d))
rmse_f = np.sqrt(mse_f)
r2_f = 1-(np.sum(d**2)/np.sum((y_test-np.mean(y_test))**2))

print("Result del calcul manual TEST:")
print("MAE:",mae_f)
print("MSE:", mse_f)
print("RMSE:", rmse_f)
print("R-Squared:", r2_f)

#
# Calculant amb les funcions 
#
print('MSE conjunt train: %.3f' % (mean_squared_error(y_test, y_test_pred)))
#
# R-Squared
#
print('R^2 conjunt train: %.3f' % (r2_score(y_test, y_test_pred)))






Result del calcul manual:
MAE: ArrDelay    40.226124
dtype: float64
MSE: ArrDelay    3692.488526
dtype: float64
RMSE: ArrDelay    60.76585
dtype: float64
R-Squared: ArrDelay    0.000766
dtype: float64
MSE conjunt train: 3692.489
R^2 conjunt train: 0.001
Result del calcul manual TEST:
MAE: ArrDelay    40.253097
dtype: float64
MSE: ArrDelay    3679.576886
dtype: float64
RMSE: ArrDelay    60.659516
dtype: float64
R-Squared: ArrDelay    0.000737
dtype: float64
MSE conjunt train: 3679.577
R^2 conjunt train: 0.001
